# **Working Process**
1. 데이터 수집하기
1. 수집데이터 전처리 NDBMS
1. Token 의 정리 (wordCloud)
1. N-Gram 의 정리 (메뉴명 PMI)
1. Clustering (군집화 묶기)
1. Token 간 관계 분석하기
1. 데이터 정리 및 모델링

# **메뉴 업데이트용 함수**
개별 작업을 구분한 뒤, 모듈로써 작업을 진행하도록 합니다
1. **크롤링용** 함수
1. 원본과 **비교용** 함수
1. **비교 후 합치는** 함수

## **1 작업용 함수 만들기**
크롤링 및 주소값을 활요한 filtering

In [1]:
# "<제목>" 추출시 lxml 로는 안되더라 왜????
import requests, re
from bs4 import BeautifulSoup
def crawMenuBs4(url_no):
    url          = "http://www.10000recipe.com/recipe/list.html?order=date&page="
    userAgent    = {"user-agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    result, resp = [], requests.request('get', url+str(url_no), headers=userAgent, timeout=5)
    dom          = BeautifulSoup(resp.text, "html.parser") 
    titles       = dom.find_all("h4", {"class":"ellipsis_title2"})
    titles       = [_.text   for _ in titles]
    for _ in dom.find_all("div", {"class":"col-xs-4"}):
        try:
            url_href = _.find("a", {"class":"thumbnail"})["href"]
            _title   = _.find("h4", {"class":"ellipsis_title2"}).text
            result.append([url_href, _title])
        except: pass
    return result
        
# 결과보기
menuList = crawMenuBs4(17)
menuList[:3]

[['/recipe/6917104', '콩국수 ~#'],
 ['/recipe/6917103', '5분이면 가능한 "건새우 볶음"'],
 ['/recipe/6917102', '간단 참치전~*']]

In [2]:
# 파일을 기준으로 결과물 필터링
import pandas as pd
def crawFileFilter(menuList, files):
    df = pd.read_csv(files, header=None)
    limitOrg = int(re.findall(r"\d+", df[0][0])[0])
    mTemp    = pd.DataFrame(menuList)
    mTemp[0] = [int(re.findall(r"\d+", _)[0]) for _ in mTemp[0]]
    mTemp    = mTemp[mTemp[0] > limitOrg]
    mTemp[0] = ["/recipe/"+str(_) for _ in mTemp[0]] # 필터링 결과 url
    return mTemp.values.tolist()

menuFilter = crawFileFilter(menuList, 'data/foodmenuRaw.csv')
menuFilter[:3]

[['/recipe/6917104', '콩국수 ~#'],
 ['/recipe/6917103', '5분이면 가능한 "건새우 볶음"'],
 ['/recipe/6917102', '간단 참치전~*']]

In [ ]:
result = []
import time
from tqdm import tqdm
for i in tqdm(range(1,20)):
    menuList   = crawMenuBs4(i)
    time.sleep(.7)
    menuFilter = crawFileFilter(menuList, 'data/foodmenuRaw.csv')
    if len(menuList) != len(menuFilter):
        result.extend(menuFilter)
        print(">>> {} limits 작업종료".format(len(menuFilter)))
        break
    else:
        result.extend(menuList)

# menuRaw = pd.read_csv('data/foodmenuRaw.csv', header=None).values.tolist()
result[:5]
pd.DataFrame(result).to_csv('data/menuNew.csv', index=None, header=None)

## **2 자료 합치기**
데이터 베이스 통합 및 정렬하기

In [50]:
from glob import glob
import pandas as pd
result   = []
filelist = glob('data/foodRecipe_*.csv', recursive=True)
for _ in sorted(filelist):
    result.extend(pd.read_csv(_, sep="|").values.tolist())
len(result),  result[:3]

(121249,
 [[6917054,
   '재료 Ingredients 계량법 안내 [재료] 베이컨 3줄 양파 1/2개 1공기 굴소스 1/2큰술 맛술 2큰술 2큰술 다진마늘 1/2큰술 쪽파 2뿌리 후춧가루 적당량 참기름 적당량',
   '재료준비 - 양파 0.5cm 두께로 썰기(눈대중으로 두툼하게) - 베이컨 2cm 두께로 썰기(8등분) - 쪽파 송송 썰기 - 마른 팬에 다진마늘, 쪽파, 양파, 베이컨 넣어 볶기 - 양파가 투명해지면 맛술, 굴소스, 물 넣어 볶기 - 국물이 자작해지면 후춧가루와 참기름 몇방울 넣어 마무리하기 - 그릇에 밥을 담고 볶은 재료 올려주면 베이컨양파덮밥 완성~!',
   '- 후춧가루와 참기름을 넣기 전 간을 보고 부족하면 굴소스 추가, 밥과 비벼먹을거라 조금 짭조롬 해야 돼요. 혹시라도 짠맛이 강하면 설탕을 조금 넣어보세요. 짠맛이 중화 된답니다.'],
  [6917053,
   '재료 Ingredients 계량법 안내 [재료] 오징어 2마리 빨강파프리카 노랑파프리카 오이 깻잎 고추장 1T 올리고당 1½T 식초 2T 요구르트 2T 통깨 1T',
   '오늘 날씨 어마어마 하더라고요. 은행 다녀왔는데 땀이 비 오듯이 줄줄줄... 대단한 날씨였죠? 냉동실에 사다 놓은 오징어로 데쳐서 먹으려고 했다가 좀 신경 쓴 요리처럼 보이게 하려고 채소를 넣어서 썰어봤어요. 이름하여 오징어야채말이입니다. 오징어 2마리 파프리카 (빨강, 노랑), 오이, 깻잎 고추장 1T, 올리고당 1½T, 식초 2T, 요구르트 2T, 통깨 1T 요즘 오징어를 자주 사다 먹고 있어요. 금징어일때 못 사 먹은 거 원 없이 사 먹고 있답니다. 2마리에 6,800원이면 사 먹을 수 있을 정도...^^;;; 오징어야채말이를 하기 위해서 오징어는 반을 갈라서 내장을 빼내고 깨끗하게 씻어주고 껍질은 벗기지 않았어요. 오징어 안쪽으로 칼집을 내주는데 한마리는 채칼로 슥삭슥삭 다른 오징어는 칼로 가로세로 칼집을 크게 넣었어요. 끓는 물에 오징어를 2분 정도 넣고 데치고 물기를 뺐어요

In [52]:
result = pd.DataFrame(result)
# result.columns = ['Url','Menu']
result.to_csv('foodRecipe.csv', index=None, header=None, sep="|")

# **RDBMS 구축하기**
## **1 데이터 살펴보기**
작업한 내용 살펴보기

In [1]:
ls data/

foodRecipe.csv  foodmens.csv  menuNames.txt  menuTitles.txt


In [21]:
# Web Page 수집함수
import requests
from lxml.html import fromstring
def crawData(url_menu):
    url        = "http://www.10000recipe.com/recipe/list.html?order=accuracy&page="
    url        = "http://www.10000recipe.com/recipe/"
    userAgent  = {"user-agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    xpath_rcp  = '//div[@id="contents_area"]//div[starts-with(@class,"cont_ingre")]//text()'
    xpath_c    = '//div[@id="contents_area"]/div[@class="view_step"]/div[starts-with(@id,"stepDiv")]//text()'
    xpath_cOld = '//div[@id="contents_area"]/div[@class="view_step"]/div[@id="oldContArea"]//text()'
    xpath_tip  = '//div[@id="contents_area"]/div[@class="view_step"]//dl[@class="view_step_tip"]//text()'
    xpath_name = '//div[@id="contents_area"]/div[@class="view2_summary"]/h3/text()'
    urlDetail  = requests.compat.urljoin(url, url_menu)
    # urlDetail  = url + str(url_menu)
    resp       = requests.request('get', urlDetail, headers=userAgent, timeout=5)
    resp_lxml  = fromstring(resp.text)
    try:
        name = resp_lxml.xpath(xpath_name)  # 레시피 데이터
        name = " ".join([_.strip()    for _ in name if len(_.strip())>1])
        recipe = resp_lxml.xpath(xpath_rcp) # 레시피 데이터
        recipe = " ".join([_.strip()  for _ in recipe if len(_.strip())>1])
        cooking = resp_lxml.xpath(xpath_c)  # 조리법 크롤링
        cooking = " ".join([_.strip() for _ in cooking if len(_.strip())>1])
        if len(cooking) < 5:                # 조리법 크롤링 (old style)
            cooking = resp_lxml.xpath(xpath_cOld)
            cooking = " ".join([_.strip() for _ in cooking if len(_.strip())>1])
        cookTip = resp_lxml.xpath(xpath_tip)
        cookTip = " ".join([_.strip()  for _ in cookTip if len(_.strip())>1])
        if len(cookTip) < 3: cookTip="None"
        # url 은 레시피 Url 숫자 저장 (압축)
        url_menu = int("".join(re.findall(r'\d+', url_menu)))
        return [url_menu, name, recipe, cooking, cookTip]
    except: return None

## **2 레시피 데이터 전처리**
작업한 내용 살펴보기

In [11]:
# import pandas as pd
# recipes    = pd.read_csv('data/foodRecipe.csv', sep="|", header=None)
# recipes[0] = [int(_)  for _ in recipes[0]] # 문자열 대신 숫자로 모두 변환

# # 중복 컬럼 줄이기 : 6917054, 6917053, 6917052, 6917051, 6917050, 6917049
# from tqdm import tqdm
# result, recipesNo = [], sorted(set(recipes[0]), reverse=True)
# for _ in tqdm(recipesNo):
#     result.append(recipes[recipes[0] == _].iloc[0,:].values.tolist())
# recipes           = pd.DataFrame(result)
# recipes.to_csv('recipe.csv', index=None, header=None, sep="|")
# recipes.head(2)

In [1]:
import pandas as pd
recipes = pd.read_csv('recipe.csv', sep="|", header=None)
recipes.columns   = ['No', 'Ingre', 'Recipe', 'Tips']
recipes.head(2)

,No,Ingre,Recipe,Tips
0,6917054,재료 Ingredients 계량법 안내 [재료] 베이컨 3줄 양파 1/2개 1공기 ...,재료준비 - 양파 0.5cm 두께로 썰기(눈대중으로 두툼하게) - 베이컨 2cm 두...,"- 후춧가루와 참기름을 넣기 전 간을 보고 부족하면 굴소스 추가, 밥과 비벼먹을거라..."
1,6917053,재료 Ingredients 계량법 안내 [재료] 오징어 2마리 빨강파프리카 노랑파프...,오늘 날씨 어마어마 하더라고요. 은행 다녀왔는데 땀이 비 오듯이 줄줄줄... 대단한...,None


In [26]:
import re, time
checkNo, err = {}, []
menus        = pd.read_csv('data/foodmens.csv', header=None)
menus[0]     = [int(re.findall(r"\d+", _)[0])   for _ in menus[0]]

from tqdm import tqdm
for _ in tqdm(recipes.No):
    try: 
        checkNo[_] = str(menus[menus[0] == _][1].values[0])
    except: 
        err.append(_)

for _ in tqdm(err): # 210개 메뉴명이 없음
    time.sleep(.5)
    checkNo[_] = crawData("/recipe/" + str(_))[1]

100%|██████████| 210/210 [02:24<00:00,  1.47it/s]


In [28]:
index_name = []
for _ in tqdm(list(recipes.No)):
    index_name.append(checkNo[_])

recipes.insert(1,'Menu', index_name)
recipes.to_csv('recipeTemp.csv', index=None, sep="|")

100%|██████████| 111250/111250 [00:00<00:00, 2099614.92it/s]


In [35]:
recipes.head(2)

,No,Menu,Ingre,Recipe,Tips
0,6917054,베이컨양파덮밥(간단 저녁메뉴),재료 Ingredients 계량법 안내 [재료] 베이컨 3줄 양파 1/2개 1공기 ...,재료준비 - 양파 0.5cm 두께로 썰기(눈대중으로 두툼하게) - 베이컨 2cm 두...,"- 후춧가루와 참기름을 넣기 전 간을 보고 부족하면 굴소스 추가, 밥과 비벼먹을거라..."
1,6917053,오징어야채말이 초고추장 찍어서 냠냠냠,재료 Ingredients 계량법 안내 [재료] 오징어 2마리 빨강파프리카 노랑파프...,오늘 날씨 어마어마 하더라고요. 은행 다녀왔는데 땀이 비 오듯이 줄줄줄... 대단한...,None


## **3 레시피 덧붙이기**
작업한 내용 살펴보기

In [53]:
import pandas as pd
recipes = pd.read_csv('recipeTemp.csv', sep="|")
recipes.head(2)

,No,Menu,Ingre,Recipe,Tips
0,6917054,베이컨양파덮밥(간단 저녁메뉴),재료 Ingredients 계량법 안내 [재료] 베이컨 3줄 양파 1/2개 1공기 ...,재료준비 - 양파 0.5cm 두께로 썰기(눈대중으로 두툼하게) - 베이컨 2cm 두...,"- 후춧가루와 참기름을 넣기 전 간을 보고 부족하면 굴소스 추가, 밥과 비벼먹을거라..."
1,6917053,오징어야채말이 초고추장 찍어서 냠냠냠,재료 Ingredients 계량법 안내 [재료] 오징어 2마리 빨강파프리카 노랑파프...,오늘 날씨 어마어마 하더라고요. 은행 다녀왔는데 땀이 비 오듯이 줄줄줄... 대단한...,None


In [59]:
menus = pd.read_csv('data/foodmenus.csv', header=None)
menus.columns = ['No', 'Menu']
menus.No = [int(re.findall(r"\d+", _)[0])  for _ in menus.No]
menus.head(2)

,No,Menu
0,6917400,호불호 없는 토마토파스타는 역시 ~! 토마토냉파스타 ★
1,6917398,"달콤 촉촉 달걀 푸딩 만들기, 만드는 법 ( 부드러운 달걀 요리 기본 원리 )"


In [58]:
menus.shape[0], len(set(menus.No)), recipes.shape[0], (menus.shape[0] - recipes.shape[0])

(122038, 111250, 10788)

In [62]:
err = []
for _ in tqdm(menus.No):
    if recipes[recipes.No == _].shape[0] == 0:
        err.append(_)

100%|██████████| 122038/122038 [01:35<00:00, 1272.66it/s]


In [65]:
len(err), err[:5]

(10995, [6917400, 6917398, 6917397, 6917396, 6917395])

In [86]:
errIndex = [menus[menus.No == _].index.values[0]  for _ in err]
menuErr  = menus.loc[errIndex]
menuErr.No = ["/recipe/" + str(_)  for _ in menuErr.No]
menuErr.head(3)

,No,Menu
0,/recipe/6917400,호불호 없는 토마토파스타는 역시 ~! 토마토냉파스타 ★
1,/recipe/6917398,"달콤 촉촉 달걀 푸딩 만들기, 만드는 법 ( 부드러운 달걀 요리 기본 원리 )"
2,/recipe/6917397,손이가요 손이가~~자꾸 손이 가는 [가지전]


In [87]:
menuErr.to_csv('menuErrNew.csv',index=None, header=None)

In [69]:
# result = []
# for _ in tqdm(err): # 10995 개 메뉴명이 없음
#     time.sleep(.5)
#     result.append(crawData("/recipe/" + str(_)))

In [70]:
menus.iloc[]

ValueError: Lengths must match to compare